下面介绍一下Zookeeper的特征。

# 1. 集合更新
Zookeeper有一个有意思的地方在于，虽然Zookeeper会被看做文件系统，但是却没有文件系统的流式操作。一方面为了保证数据的数据全局一致性，另一方面另外这个文件系统很少存数据整体很小，所以Zookeeper是整体读整体写的没有打开和关闭操作。

# 2. 观察触发器
在使用exists、getChildren、getData这些函数上可以设置观察器，在create、delete、setData操作时触发。触发事件类型：

|设置观察的操作|创建znode|创建子节点|删除znode|删除子节点|setData|
|---|---|---|---|---|---|
|exists|NodeCreated| - |NodeDeleted| - |NodeDataChanged|
|getData| - | - |NodeDeleted| - |NodeDataChanged|
|getChildren| - |NodeChildrenChanged|NodeDeleted|NodeChildrenChanged| - |

另外观察触发器，只会触发一次，当触发完成之后观察器就会失效。如果我们需要一直保持有观察器的状态则需要再创建一个观察器。

## 2.1 demo：只触发一次观察
```
package com.demo;

import org.apache.zookeeper.*;

import java.io.IOException;
import java.util.concurrent.CountDownLatch;

public class ConnectWatcher implements Watcher {

    protected static final int SESSION_TTMEOUT = 5000;

    protected ZooKeeper zk;
    private CountDownLatch connectedSignal = new CountDownLatch(1);

    public void connect(String hosts) throws IOException, InterruptedException {
        zk = new ZooKeeper(hosts, SESSION_TTMEOUT, this);
        connectedSignal.await();
    }

    /**
     * 获取事件之后的回调函数
     */
    public void process(WatchedEvent watchedEvent) {
        if(watchedEvent.getState() == Event.KeeperState.SyncConnected) {
            connectedSignal.countDown();
        }
    }

    public void close() throws InterruptedException {
        zk.close();
    }

}
```
## 2.2 demo：一直存在观察
```
package com.demo;

import org.apache.zookeeper.KeeperException;
import org.apache.zookeeper.WatchedEvent;
import org.apache.zookeeper.Watcher;
import org.apache.zookeeper.ZooKeeper;

import java.io.IOException;
import java.util.List;
import java.util.concurrent.CountDownLatch;

public class ConnectMultiWatcher implements Watcher {

    protected static final int SESSION_TTMEOUT = 5000;

    protected ZooKeeper zk;
    private CountDownLatch connectedSignal = new CountDownLatch(1);

    public void connect(String hosts) throws IOException, InterruptedException {
        zk = new ZooKeeper(hosts, SESSION_TTMEOUT, this);
        connectedSignal.await();
    }

    /**
     * 获取事件之后的回调函数
     */
    public void process(WatchedEvent watchedEvent) {
        try {
            System.out.println( watchedEvent.getPath() + "\t\t" + watchedEvent.getType());
            zk.getChildren("/", true);
        } catch (KeeperException e) {
            e.printStackTrace();
        } catch (InterruptedException e) {
            e.printStackTrace();
        }
    }

    public void close() throws InterruptedException {
        zk.close();
    }

}
```
# 3. ACL(访问控制列表,Access Control List)
Zookeeper在创建znode的时候可以设置谁可以对他执行操作。具体内容略...